Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.
### Licensed under the Apache License, Version 2.0 (the "License");  
You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# PyTorch BERT FineTuning Example on Habana Gaudi

This Jupyter Notebook example demonstrates how to finetune BERT on Habana Gaudi device with PyTorch framework. The pretrained model will be downloaded from HuggingFace, and finetuned with SQuAD dataset.

In [ ]:
%cd /root

## Setup

Let's clone Habana `Model-References` repository to this image and add it to PYTHONPATH.

In [ ]:
!git clone https://github.com/habanaai/Model-References

In [ ]:
!export PYTHONPATH=/root/Model-References:$PYTHONPATH
!export PYTHON=/usr/bin/python3.8

In [ ]:
%cd /root/Model-References/PyTorch/nlp/bert

Next, we need to install all the Python packages that BERT depends on.  

In [ ]:
!python3 -m pip install --quiet -r requirements.txt

Download the Vocab File.  This folder is using vocab.txt only

In [ ]:
!wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-10_H-768_A-12.zip
!unzip uncased_L-10_H-768_A-12.zip

Download the checkpoints from the Habana Vault

In [ ]:
!wget https://vault.habana.ai/artifactory/pretrained-models/checkpoints/1.9.0/PT/BertL-PT/BertL-PT-PyTorch-1.13.1-1.9.0-580-32n-checkpoint.tar.gz
!tar -xvzf BertL-PT-PyTorch-1.13.1-1.9.0-580-32n-checkpoint.tar.gz

Download and pre-process Squad Dataset(V1.1)

In [ ]:
!sh ./data/squad/squad_download.sh

In [ ]:
!pwd

## Training on 1 HPU

After all the dependant Python packages are installed, let's launch BERT base finetuning with SQuAD dataset on a single HPU:

We see that the original BERT model is now Fine Tuned with the SQuAD dataset 

```
      $PYTHON run_squad.py --do_train --bert_model=bert-large-uncased --config_file=./bert_config.json \
```

In [ ]:
%cd /root/Model-References/PyTorch/nlp/bert
%mkdir /tmp/log_directory

In [ ]:
!python3 run_squad.py --do_train --bert_model=bert-large-uncased --config_file=./bert_config.json \
 --use_habana --use_fused_adam --do_lower_case --output_dir=/tmp/results/checkpoints \
 --json-summary=/tmp/log_directory/dllogger.json \
 --train_batch_size=12 --predict_batch_size=8 --seed=1 --max_seq_length=384 \
 --doc_stride=128 --max_steps=-1   --learning_rate=3e-5 --num_train_epochs=2  \
 --vocab_file=vocab.txt \
 --init_checkpoint=ckpt_8601.pt  \
 --train_file=./v1.1/train-v1.1.json \
 --skip_cache --do_predict  \
 --predict_file=./v1.1/dev-v1.1.json \
 --do_eval --eval_script=./v1.1/evaluate-v1.1.py --log_freq 20 

**From the logs above, we can see the finetuning throughput for BERT base on 1 HPU is over 100 samples/second.**